# ABSCAL Example Notebook

## WFC3 IR Grism

This notebook will take you through the steps of downloading sample MAST data and running the WFC3 IR grism abscal scripts on that data.

### Set up Python Environment

This step imports the python modules used by this script.

In [ ]:
import glob
import os
import numpy as np
import shutil

from astroquery.mast import Observations
from pathlib import Path
from tempfile import TemporaryDirectory

from abscal.wfc3.preprocess_table_create import populate_table
from abscal.wfc3.reduce_grism_coadd import coadd
from abscal.wfc3.reduce_grism_wavelength import wlmeas, wlmake

%matplotlib inline

work_dir = os.getcwd()

### Optional: Set up temporary directory for data

By default, notebooks store downloaded files (etc.) in the directory in which they are running. If you don't wish to do this (or don't have access to that directory), you can set up a temporary directory and store data in it.

In [ ]:
# Set this flag to True if you wish to use a temporary directory
use_temporary_dir = False

# Set this flag if you want to define a custom directory to work in
use_custom_dir = True

if use_temporary_dir:
    data_dir = TemporaryDirectory()
    os.chdir(data_dir.name)
    work_dir = data_dir.name

if use_custom_dir:
    work_dir = "/Users/york/Projects/abscal/examples/notebook_dev"

print("Storing data in {}".format(work_dir))

### Optional: Download input data from MAST

This notebook is designed to be run with any WFC3 IR grism data (although planetary nebula observations of a known target will be required for the wavelength steps). In order to simply see how these scripts work with example data, or to test their operation, you can use a set of example data.

The next cell defines a function that will download all non-HLA data from a specific HST observing program (i.e. data whose observation ID does not begin with "hst_"), and copy the downloaded files into a single directory (here, the same directory where the notebook is running). This function may be more generally useful for retrieving observations from MAST, and can be copied and used separately (or modified to suit) as long as the following import statements are included:

    from astroquery.mast import Observations
    import os
    import shutil

The following cell will download two sets of example data (a flux calibration target and a planetary nebula target) from MAST. In particular, it will download program 15587 for flux calibration, and 13582 for planetary nebula data.

If you already have downloaded data with which you want to work, these cells can be skipped entirely.

In [ ]:
def download_mast_program(proposal_id, download_dir, skip_existing_files=True):
    flux_table = Observations.query_criteria(proposal_id=proposal_id)
    obs_mask = [x[:4] != 'hst_' for x in flux_table['obs_id']]
    obs_filter = [id for id in flux_table['obs_id'] if id[:4] != 'hst_']
    flux_table = flux_table[obs_mask]
    obs_ids = flux_table['obsid']
    if skip_existing_files:
        i = 0
        while i < len(obs_filter):
            # This is an idiom for going through a list and potentially removing items from it. If you're going 
            # through a list in a for loop, the result of removing items from the list during the loop is not 
            # well-defined, so this method is used instead.
            if len(glob.glob(os.path.join(download_dir, obs_filter[i]+"*"))) > 0:
                obs_filter.remove(obs_filter[i])
            else:
                i += 1
    data_products = Observations.get_product_list(obs_ids)
    if len(data_products) > 0 and len(obs_filter) > 0:
        manifest = Observations.download_products(data_products, download_dir=download_dir, extension=["fits"], 
                                                  obs_id=obs_filter)

        for file_name in manifest['Local Path']:
            base_file = os.path.basename(file_name)
            print("Copying {}".format(base_file))
            shutil.copy(os.path.join(download_dir, file_name), os.path.join(download_dir, base_file))

In [ ]:
# Download the planetary nebula program
download_mast_program(13582, work_dir)

# Download the flux calibration program
download_mast_program(15587, work_dir)

### Set up the initial data table

This cell will cet up a data table of all WFC3 data in the current directory. Note that, in succeeding steps, only the IR grism data will actually be reduced (except that filter data taken at the same position and POSTARG during the same visit will be used, if available, to derive an initial location of the grism zeroth-order), so the presence of data other than IR grism data will not confuse the remaining scripts.

The populate_table function called below can take a variety of arguments. In particular, if you have an existing table of observations (in the form of an AbscalDataTable), you can pass in that table and add any additional observations to that table. Also, the function can take arbitrary keyword arguments, which are currently used to set whether to use verbose output, and whether to output an IDL-compatible data table, in the future there may be additional settable parameters that will affect the way that data is ingested into a table.

In [ ]:
verbose_output = True

data_table = populate_table(verbose=True, search_dirs=work_dir)

Now that there's a data table, let's take a look at what's in it (and what its features are). The data table holds a list of observations along with metadata describing the date, program, visit, grism or filter used, and other exposure parameters. There are also a number of columns related to the current abscal run, of which only two (the path at which the observation can be found, and the file name used to obtain the other metadata) are currently filled in. The AbscalDataTable class subclasses the Astropy table class, so for or less anything from the Astropy table documentationi at <https://docs.astropy.org/en/stable/table/access_table.html> can be used here.

### Reduce the WFC3 Grism Data

This cell will take the data table from `populate_table()` and reduce all of the grism exposures in it. `coadd()` function allows for many default parameters to be reset at runtime.

<div class="alert alert-block alert-warning">
    <strong>NOTE:</strong> Jupyter notebooks do not allow blocking calls in the middle of cells. When run as a 
    script, ABSCAL uses blocking figures with text-input boxes to allow user interaction. As such, when running
    ABSCAL from a notebook, there is currently no way to use interactive elements.
</div>

In [ ]:
reduced_table = coadd(data_table, out_dir=work_dir, verbose=True, plots=True)

The main difference between `data_table` and `reduced_table` is that the latter has many more parameters filled in (zeroth order image location, extracted spectrum file, co-added file, etc.

### Measure Wavelength Fit

This cell takes the planetary nebula exposures in reduced_table, and finds the location of a set of emission lines. When run interactively, this allows the user to override fit locations and choose to reject fits.

<div class="alert alert-block alert-warning">
    <strong>NOTE:</strong> The "notebook" flag is required in order to display non-interactive plots without being
    trapped in an endless loop.
</div>

In [ ]:
wavelength_table = wlmeas(reduced_table, verbose=True, plots=True, out_dir=work_dir, notebook=True)

The wavelength_table is a standard Astropy table rather than an AbscalDataTable, and stores only information on the derived line positions.

### Generate Wavelength Solution

This cell takes the results of the wavelength fit above, and uses it to derive a full-detector wavelength solution based on position relative to the zeroth order.

In [ ]:
fit_table = wlmake(reduced_table, wavelength_table, verbose=True, plots=True, out_dir=work_dir)

The fit_table contains only the fit parameters, and does not preserve any of the standard error information.